In [ ]:
!pip install autokeras
!pip install tensorboardX
!pip install mosaicml
import tensorflow as tf
import autokeras as ak
from keras.datasets import cifar10

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import composer.functional as cf

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Take only 5000 samples from the training set
num_samples = 5000
train_images = train_images[:num_samples]
train_labels = train_labels[:num_samples]

# Take only 5000 samples from the test set
test_images = test_images[:num_samples]
test_labels = test_labels[:num_samples]

# The 'train_images', 'train_labels', 'test_images', and 'test_labels' now contain only 5000 samples each
# Normalize the pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0


In [ ]:

class LayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        input_node = tf.nest.flatten(inputs)[0]

        # First Dense Layer
        layer1 = tf.keras.layers.Dense(units=hp.Int("Layer_1", min_value=32, max_value=1024, step=32),activation=hp.Choice("activation_1", values=["relu","softmax", "tanh","sigmoid"]))
        output_node1 = layer1(input_node)

        # Second Dense Layer
        layer2 = tf.keras.layers.Dense(units=hp.Int("Layer_2", min_value=32, max_value=1024, step=32),activation=hp.Choice("activation_2", values=["relu","softmax", "tanh","sigmoid"]))
        output_node2 = layer2(output_node1)

        return output_node2

In [ ]:
class LogValidationLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Access the validation loss and other metrics from logs
        val_loss = logs['val_loss']
        # You can also access other metrics such as accuracy using logs['val_accuracy']

        # Store the validation loss in a file or data structure of your choice
        # Here, we will append it to a list
        validation_losses.append(val_loss)

In [ ]:
validation_losses = []

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="hyperband",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=100,
    seed=42,
    directory="/my/directory",
    project_name='my_custom_image_classifier',
    objective="val_loss"
)
auto_model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, callbacks=[LogValidationLossCallback()])
truncated_list = validation_losses[:-10]
sublists = [truncated_list[i:i+10] for i in range(0, len(truncated_list), 10)]


Trial 55 Complete [00h 00m 21s]
val_loss: 2.081728219985962

Best val_loss So Far: 1.586416244506836
Total elapsed time: 00h 29m 40s

Search: Running Trial #56

Value             |Best Value So Far |Hyperparameter
128               |128               |layer_block_1/Layer_1
sigmoid           |relu              |layer_block_1/activation_1
32                |32                |layer_block_1/Layer_2
relu              |relu              |layer_block_1/activation_2
flatten           |flatten           |classification_head_1/spatial_reduction_1/reduction_type
0.25              |0.25              |classification_head_1/dropout
adam_weight_decay |adam_weight_decay |optimizer
0.001             |0.001             |learning_rate

Epoch 1/10
157/157 [==============================] - 3s 12ms/step - loss: 2.2638 - accuracy: 0.1522 - val_loss: 2.1281 - val_accuracy: 0.2310
Epoch 2/10
157/157 [==============================] - 2s 14ms/step - loss: 2.0641 - accuracy: 0.2674 - val_loss: 2.0356 - val_acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

tuner = auto_model.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = auto_model.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)
df = pd.DataFrame(trials_hparams)
df['val_Loss']=score


In [ ]:
df.to_csv("/content/drive/My Drive/hyperband.csv", index=False)

In [ ]:
import csv
csv_file_path = '/content/drive/My Drive/hyperbandepochs.csv'

# Write the list of lists to the CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(sublists)

In [ ]:
for i, sublist in enumerate(sublists):
    # Filter values greater than 2.5
    filtered_values = [value if value <= 2.35 else None for value in sublist]

    # Plot the filtered sublist with a label
    plt.plot(filtered_values, label=f'Sublist {i+1}')

# Add labels and title
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Graph of Sublists (Ignoring Values > 2.5)")

# Add a legend
plt.legend()

# Show the graph
plt.show()

In [ ]:
validation_losses = []

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="greedy",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10,
    seed=42,
    directory="/my/directory",
    project_name='my_custom_image_classifier',
    objective="val_loss"
)
auto_model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, callbacks=[LogValidationLossCallback()])
truncated_list = validation_losses[:-10]
sublists = [truncated_list[i:i+10] for i in range(0, len(truncated_list), 10)]
for i, sublist in enumerate(sublists):
    # Filter values greater than 2.5
    filtered_values = [value if value <= 2.35 else None for value in sublist]

    # Plot the filtered sublist with a label
    plt.plot(filtered_values, label=f'Sublist {i+1}')

# Add labels and title
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Graph of Sublists (Ignoring Values > 2.5)")

# Add a legend
plt.legend()

# Show the graph
plt.show()

In [ ]:
validation_losses = []

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="hyperband",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10,
    seed=42,
    directory="/my/directory",
    project_name='my_custom_image_classifier',
    objective="val_loss"
)
auto_model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, callbacks=[LogValidationLossCallback()])
truncated_list = validation_losses[:-10]
sublists = [truncated_list[i:i+10] for i in range(0, len(truncated_list), 10)]
for i, sublist in enumerate(sublists):
    # Filter values greater than 2.5
    filtered_values = [value if value <= 2.35 else None for value in sublist]

    # Plot the filtered sublist with a label
    plt.plot(filtered_values, label=f'Sublist {i+1}')

# Add labels and title
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Graph of Sublists (Ignoring Values > 2.5)")

# Add a legend
plt.legend()

# Show the graph
plt.show()

In [ ]:
validation_losses = []

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="random",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10,
    seed=42,
    directory="/my/directory",
    project_name='my_custom_image_classifier',
    objective="val_loss"
)
auto_model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, callbacks=[LogValidationLossCallback()])
truncated_list = validation_losses[:-10]
sublists = [truncated_list[i:i+10] for i in range(0, len(truncated_list), 10)]
for i, sublist in enumerate(sublists):
    # Filter values greater than 2.5
    filtered_values = [value if value <= 2.35 else None for value in sublist]

    # Plot the filtered sublist with a label
    plt.plot(filtered_values, label=f'Sublist {i+1}')

# Add labels and title
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Graph of Sublists (Ignoring Values > 2.5)")

# Add a legend
plt.legend()

# Show the graph
plt.show()